In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
import os, sys
sys.path.append('..')
from training.Helpers import Take_data,Test_model
pd.set_option('display.max_columns', None)

In [2]:
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [3]:
m = xgb.XGBRFClassifier().n_estimators

In [4]:
params = {
 "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30,0.4,0.6],
 "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15,16,17,18,19,20,25],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [5]:
model = xgb.XGBRFClassifier()
rs_model=RandomizedSearchCV(model,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=2,verbose=3)

In [6]:
df = pd.read_csv('../data/original_data/train.csv')

In [7]:
df.orig_destination_distance.fillna(1970)

0           2234.2641
1           2234.2641
2           2234.2641
3            913.1932
4            913.6259
              ...    
37670288    2348.4075
37670289     679.6104
37670290     668.1768
37670291     106.4274
37670292     106.4274
Name: orig_destination_distance, Length: 37670293, dtype: float64

In [8]:
pd.read_csv(os.path.join('../data/datasets/train',f'sample_{str(1000)}.csv')).drop(columns=['Unnamed: 0'])

,0
0,19702595
1,19514203
2,33199442
3,34864924
4,25705435
...,...
998,6467675
999,33634761
1000,22251977
1001,23703433


In [9]:
x,y = Take_data(sample_size=1000,df=df)

C:\Users\macie\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [39]:
def Preprocessing(data):
    data.drop(columns=['user_id'],inplace=True) 
    for column_name in ['date_time','srch_ci','srch_co']:
        data[column_name + '_month'] = pd.DatetimeIndex(data[column_name]).month
        data[column_name +'_day'] = pd.DatetimeIndex(data[column_name]).day
        data.drop(columns=[column_name],inplace=True)
        
    data['orig_destination_distance'] = data.orig_destination_distance.fillna(1970)
    # sc = load('../parameters/sc_distance.bin')
    # print(len(data.orig_destination_distance))
    # _ = sc.transform(data.orig_destination_distance.values.reshape(-1,1))
    # data['orig_destination_distance'] = _ 
    
    x = data.drop(columns=['hotel_cluster'])
    destinations = pd.read_csv('../data/original_data/destinations.csv')
    x = x.join(destinations,on='srch_destination_id',how='inner',rsuffix='_right').drop(columns=['srch_destination_id_right','srch_destination_id'])
    y = data.hotel_cluster
    return x,y
    

def Take_data(sample_size = 10000,df=None):
    #reading train data
    if isinstance(df,type(None)):
        df = pd.read_csv('../data/original_data/train.csv')
        test_id = pd.read_csv('../data/datasets/test/test_id.csv')
        df = df.loc[~df.index.isin(test_id)]
    
    #sampling or reading previosusly sampled data
    if f'sample_{str(sample_size)}.csv' in os.listdir('../data/datasets/train'):
        sample_id = pd.read_csv(os.path.join('../data/datasets/train',f'sample_{str(sample_size)}.csv')).drop(columns=['Unnamed: 0'])
        #FIXME
        sample = df.loc[df.index.isin(sample_id)]
        print(len(sample_id),df.index.isin(sample_id))
    else:
        sample = df.groupby('hotel_cluster').apply(lambda x: x.sample(frac=(sample_size/37670293))).droplevel(level=0)
    #preprocessing    
    x,y = Preprocessing(data=sample)
    if f'sample_{str(sample_size)}.csv' not in os.listdir('../data/datasets/train'):
        pd.Series(x.index).to_csv(f'../data/datasets/train/sample_{str(sample_size)}.csv')
    return x,y

def Test_model(model):
    df = pd.read_csv('../data/original_data/train.csv')
    test_id = pd.read_csv('../data/datasets/test/test_id.csv')
    df = df.loc[df.index.isin(test_id)]
    
    x,y = Preprocessing(df)
    predicted = model.predict(x)
    return metrics(predicted,y)
    
def metrics(pred,true):
    from sklearn.metrics import accuracy_score,balanced_accuracy_score,recall_score,balanced_accuracy_score
    metrics = [accuracy_score,balanced_accuracy_score,recall_score,balanced_accuracy_score]
    result = {}
    for metric in metrics:
        result[metric.__name__] = metric(true,pred)
    return result      

In [40]:
x1, y1 = Take_data(sample_size=1000,df=df)

999 37670293


C:\Users\macie\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [38]:
x1

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,date_time_month,date_time_day,srch_ci_month,srch_ci_day,srch_co_month,srch_co_day,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d50,d51,d52,d53,d54,d55,d56,d57,d58,d59,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d70,d71,d72,d73,d74,d75,d76,d77,d78,d79,d80,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90,d91,d92,d93,d94,d95,d96,d97,d98,d99,d100,d101,d102,d103,d104,d105,d106,d107,d108,d109,d110,d111,d112,d113,d114,d115,d116,d117,d118,d119,d120,d121,d122,d123,d124,d125,d126,d127,d128,d129,d130,d131,d132,d133,d134,d135,d136,d137,d138,d139,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149


In [30]:
x

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,date_time_month,date_time_day,srch_ci_month,srch_ci_day,srch_co_month,srch_co_day,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d50,d51,d52,d53,d54,d55,d56,d57,d58,d59,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d70,d71,d72,d73,d74,d75,d76,d77,d78,d79,d80,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90,d91,d92,d93,d94,d95,d96,d97,d98,d99,d100,d101,d102,d103,d104,d105,d106,d107,d108,d109,d110,d111,d112,d113,d114,d115,d116,d117,d118,d119,d120,d121,d122,d123,d124,d125,d126,d127,d128,d129,d130,d131,d132,d133,d134,d135,d136,d137,d138,d139,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
10717623,11,3,205,385,47462,1222.9993,0,1,9,2,3,2,1,0,1,2,50,682,1,30,NaN,NaN,NaN,NaN,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.060932,-2.190735,-2.190735,-2.190735,-2.190735,-2.153942,-2.190735,-2.175641,-2.157985,-2.190735,-2.161055,-2.164662,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.139764,-2.190735,-2.149187,-2.190735,-2.190735,-2.085479,-2.190735,-2.190735,-2.035550,-2.190735,-2.190735,-2.190735,-2.190735,-1.823729,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.143320,-2.190735,-2.190735,-2.190735,-2.183122,-2.190735,-2.063798,-2.190735,-2.190735,-2.190735,-2.175641,-2.190735,-2.175641,-2.168287,-2.190643,-2.169336,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.136436,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.148252,-2.190735,-2.190735,-2.190735,-2.190735,-2.183122,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.060334,-2.190735,-1.882920,-2.190735,-2.183122,-2.136728,-2.190735,-2.190735,-2.190735,-2.190735,-2.175641,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.141909,-2.190735,-2.183122,-2.190735,-2.190735,-2.190735,-2.139131,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.183122,-2.183122,-2.190735,-2.190735,-2.190735,-2.190735,-2.183122,-2.183122,-2.190735,-2.190735,-2.175641,-2.158300,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.124184,-2.175641,-2.190735,-2.190735,-2.190735,-2.190735,-2.168287,-2.175479,-2.190735,-2.190735,-2.190735
8763975,34,3,231,102,35582,1970.0000,0,0,9,2,0,1,1,0,2,2,50,682,7,30,8.0,30.0,9.0,2.0,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.060932,-2.190735,-2.190735,-2.190735,-2.190735,-2.153942,-2.190735,-2.175641,-2.157985,-2.190735,-2.161055,-2.164662,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.139764,-2.190735,-2.149187,-2.190735,-2.190735,-2.085479,-2.190735,-2.190735,-2.035550,-2.190735,-2.190735,-2.190735,-2.190735,-1.823729,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.143320,-2.190735,-2.190735,-2.190735,-2.183122,-2.190735,-2.063798,-2.190735,-2.190735,-2.190735,-2.175641,-2.190735,-2.175641,-2.168287,-2.190643,-2.169336,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.136436,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.148252,-2.190735,-2.190735,-2.190735,-2.190735,-2.183122,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.060334,-2.190735,-1.882920,-2.190735,-2.183122,-2.136728,-2.190735,-2.190735,-2.190735,-2.190735,-2.175641,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.141909,-2.190735,-2.183122,-2.190735,-2.190735,-2.190735,-2.139131,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.190735,-2.183122,-2.183122,-2.190735,-2.190735,-2.190735,-2.190735,-2.183122,-2.183122,-2.190735,-2.190735,-2.175641,-2.158300,-2.190735,-2.190735,-2.1907

In [25]:
pd.Series(x.index).to_csv(f'../data/datasets/train/sample_{str(1000)}.csv')

In [8]:
destinations = pd.read_csv('../data/original_data/destinations.csv')

In [9]:
destinations

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62101,64989,-2.145876,-2.158016,-2.216621,-2.158016,-2.006209,-2.102505,-2.216621,-2.157203,-2.216621,...,-2.216621,-2.158016,-2.158016,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.216621,-2.152024
62102,64990,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.176802,-2.203479,-2.203479,-2.203479,...,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.203479,-2.008885
62103,64991,-2.190567,-2.208308,-2.208308,-2.208308,-2.208308,-2.135493,-2.208308,-2.208308,-1.909808,...,-2.188935,-2.208308,-2.175594,-2.208308,-2.208308,-2.199347,-2.208308,-2.208308,-2.208308,-2.208308
62104,64992,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,...,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686,-2.173686


In [22]:
x.join(destinations,on='srch_destination_id',how='inner',rsuffix='_right').drop(columns=['srch_destination_id_right','srch_destination_id'])

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,date_time_month,date_time_day,srch_ci_month,srch_ci_day,srch_co_month,srch_co_day,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d50,d51,d52,d53,d54,d55,d56,d57,d58,d59,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d70,d71,d72,d73,d74,d75,d76,d77,d78,d79,d80,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90,d91,d92,d93,d94,d95,d96,d97,d98,d99,d100,d101,d102,d103,d104,d105,d106,d107,d108,d109,d110,d111,d112,d113,d114,d115,d116,d117,d118,d119,d120,d121,d122,d123,d124,d125,d126,d127,d128,d129,d130,d131,d132,d133,d134,d135,d136,d137,d138,d139,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
19702595,2,3,66,174,47632,1243.049400,0,1,0,2,0,1,1,1,1,4,8,1532,4,28,5.0,29.0,6.0,2.0,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.155706,-2.174610,-2.174610,-2.174610,-2.155706,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.155706,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610
21436991,2,3,66,174,26232,1571.146000,1,1,0,2,0,1,1,0,1,4,8,1532,5,5,8.0,6.0,8.0,11.0,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.155706,-2.174610,-2.174610,-2.174610,-2.155706,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.137590,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.155706,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174610,-2.174

In [5]:
res = []
params = {
    "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30,0.4,0.6],
    "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15,16,17,18,19,20,25],
    "min_child_weight" : [ 1, 3, 5, 7 ],
    "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]}

for sample_size in [1000,2000,5000,8000,10000,15000,20000,30000,50000,80000,100000,200000,500000,800000,1000000]:
    print(f'sample: {sample_size}')
    x,y = Take_data(sample_size=sample_size)
    model = xgb.XGBRFClassifier()
    rs_model=RandomizedSearchCV(model,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=2,verbose=3)
    rs_model.fit(x,y)
    res[sample_size]=Test_model(rs_model)
    pd.DataFrame(res).to_pickle(f'../results/res_XGBRFC_{sample_size}.pkl')
    

KeyboardInterrupt: 

In [30]:
x,y = Take_data(1000)

In [31]:
x

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_mobile,is_package,channel,srch_adults_cnt,...,cnt,hotel_continent,hotel_country,hotel_market,date_time_month,date_time_day,srch_ci_month,srch_ci_day,srch_co_month,srch_co_day
12501982,17,1,133,25,17430,1933.869761,0,0,5,2,...,1,2,50,214,7,24,8.0,14.0,8.0,19.0
879526,2,3,66,174,24103,2558.118600,0,0,9,4,...,1,2,50,212,9,5,9.0,6.0,9.0,10.0
22596580,2,3,66,174,46432,2463.253000,0,0,0,2,...,2,2,50,212,9,16,10.0,19.0,10.0,23.0
22192042,2,3,66,348,26037,1550.317700,0,1,9,3,...,1,4,8,110,11,18,2.0,17.0,2.0,21.0
9867432,2,3,66,462,14703,2595.876700,0,1,9,2,...,3,2,50,636,10,17,11.0,13.0,11.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36650114,11,3,205,385,47462,644.317200,0,1,9,2,...,2,2,50,637,7,23,11.0,6.0,11.0,10.0
27329015,24,2,3,66,28951,1933.869761,0,0,9,2,...,1,3,82,230,11,8,7.0,23.0,7.0,26.0
36516337,11,3,205,385,46963,3244.005500,1,0,9,2,...,2,6,70,19,11,24,6.0,1.0,6.0,26.0
25906878,24,2,3,64,33550,1933.869761,0,0,9,2,...,1,3,126,232,4,7,5.0,29.0,6.0,1.0


In [49]:
rs_model.fit(x,y)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


c:\Users\macie\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
c:\Users\macie\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(
C:\Users\macie\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:45:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=2,
                   estimator=XGBRFClassifier(base_score=None, booster=None,
                                             colsample_bylevel=None,
                                             colsample_bytree=None, gamma=None,
                                             gpu_id=None,
                                             importance_type='gain',
                                             interaction_constraints=None,
                                             max_delta_step=None,
                                             max_depth=None,
                                             min_child_weight=None, missing=nan,
                                             monotone_constraints=None,
                                             n_estimators=100, n_jobs=None,
                                             num_parallel_tree=None,
                                             obje...
                                             random_state=None, reg_al

In [1]:
rs_model.best_estimator_.get_params()

NameError: name 'rs_model' is not defined